# KMeans

### Libraries

In [ ]:
!pip install pandas==1.5.3
!pip install tsfel
!pip3 install --upgrade --no-cache-dir gdown       # support for download a large file from Google Drive
!pip install numpy>=1.19.5
!pip install scikit-learn>=0.24.1

### Download dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# unzip from drive
!unzip /content/drive/MyDrive/Colab_MLA/MLA_Project/csv_20220811.zip -d /content/

In [ ]:
# or Download from link
import os, sys
# https://drive.google.com/file/d/1Fn_KVRpwLedTYU1QgfVCRtkvo1hf_9GB/view?usp=sharing first account
# https://drive.google.com/file/d/1P8pCKLI-64_HT91Oqid4RUGtZCUht2c-/view?usp=sharing second account

if not os.path.isfile('/content/csv_20220811.zip'):
  !gdown 1P8pCKLI-64_HT91Oqid4RUGtZCUht2c-
  !jar xvf  "/content/csv_20220811.zip"

if not os.path.isdir('/content/csv_20220811'):
  print("Dataset doesn't exist")

In [4]:
import os, sys
import time
import warnings
import datetime
import torch
import tsfel
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import logging
import pickle
import random as rn
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import BaseEstimator, OutlierMixin
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn import metrics
from google.colab import files
from sklearn.metrics import roc_auc_score, average_precision_score, top_k_accuracy_score, f1_score, roc_curve, auc, precision_recall_curve

### Data Loading

In [5]:
ROOTDIR_DATASET_NORMAL = "/content/csv_20220811"
plt.style.use("Solarize_Light2") # Set style for matplotlib

##### Loading metadata

In [6]:
def get_metadata(filepaths_csv, filepaths_meta, action2int=None, delimiter=";"):
  dfs_meta = list()
  for filepath in filepaths_meta:                                   # read filepath (0, 2, 3, 4)
        df_m = pd.read_csv(filepath, sep=delimiter)                 # read csv files
        df_m.str_repr = df_m.str_repr.str.replace('True', 'true')   # replace True with true
        df_m['filepath'] = filepath                                 # create the 'filepath' column
        dfs_meta.append(df_m)                                       # add the corresponding dataframe
  df_meta = pd.concat(dfs_meta)                                     # concatenate all dataframes

  df_meta.index = pd.to_datetime(df_meta.init_timestamp.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")                        # convert numerical index in time index
  df_meta['completed_timestamp'] = pd.to_datetime(df_meta.completed_timestamp.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")  # change format of completed_timestamp
  df_meta['init_timestamp'] = pd.to_datetime(df_meta.init_timestamp.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")            # change format of init_timestamp

  actions = df_meta.str_repr.unique()                                             # due to the concat of before we know take the actions removing the duplicate

  dfs = [pd.read_csv(filepath_csv, sep=";") for filepath_csv in filepaths_csv]    # read the train/test datas
  df = pd.concat(dfs)                                                             # concat the data like before
  df = df.sort_index(axis=1)                                                      # sort columns by name

  df.index = pd.to_datetime(df.time.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")        # set timestamp as index
  columns_to_drop = [column for column in df.columns if "Abb" in column or "Temperature" in column] # remove uselesse columns
  df.drop(["machine_nameKuka Robot_export_active_energy", "machine_nameKuka Robot_import_reactive_energy"] + columns_to_drop, axis=1, inplace=True)

  df_action = list()        # take the actions
  for action in actions:    # loop for each action (30)
      for index, row in df_meta[df_meta.str_repr == action].iterrows(): # get index and row from metadata where the actions are the same
          start = row['init_timestamp']         # start
          end = row['completed_timestamp']      # end
          df_tmp = df.loc[start: end].copy()    # temporary dataframe
          df_tmp['action'] = action             # get action
          df_tmp['duration'] = str((row['completed_timestamp'] - row['init_timestamp']).total_seconds())    # life of the action (it's not a feature)
          df_action.append(df_tmp)
  df_action = pd.concat(df_action, ignore_index=True)     # concatenate the actions
  df_action.index = pd.to_datetime(df_action.time.astype('datetime64[ms]'), format="%Y-%m-%dT%H:%M:%S.%f")   # set the time as index
  df_action = df_action[~df_action.index.duplicated(keep='first')]     # keep the duplicate

  df = df.dropna(axis=0)                  # remove NaNs from Df (34275, 56)
  df_action = df_action.dropna(axis=0)    # (33063, 58)

  if action2int is None:        # map the actions to integer --> 30 action - 30 indexes from 1 to 30
      action2int = dict()
      j = 1
      for label in df_action.action.unique():
          action2int[label] = j
          j += 1

  df_merged = df.merge(df_action[['action']], left_index=True, right_index=True, how="left")  # (34275, 57)
  df_idle = df_merged[df_merged['action'].isna()].copy()    # (1212, 57)
  df_idle['action'] = 'idle'
  df_idle['duration'] = df_action.duration.values.astype(float).mean().astype(str)
  df_action = pd.concat([df_action, df_idle]) # (34275, 58)
  action2int['idle'] = 0
  return df_action, df, df_meta, action2int

##### Loading test and train dataset

In [7]:
frequency = 1    # 1 10 100 200 Hz - life {1: 10, 10: 1, 100: 0.1, 200: 0.05}

filepath_csv_test = [os.path.join(ROOTDIR_DATASET_NORMAL, f"rec{r}_collision_20220811_rbtc_{1/frequency}s.csv") for r in [1, 5]]        # read data with anomalies
filepath_meta_test = [os.path.join(ROOTDIR_DATASET_NORMAL, f"rec{r}_collision_20220811_rbtc_{1/frequency}s.metadata") for r in[1, 5]]

filepath_csv_train = [os.path.join(ROOTDIR_DATASET_NORMAL, f"rec{r}_20220811_rbtc_{1/frequency}s.csv") for r in [0, 2, 3, 4]]           # read non-anomalous data
filepath_meta_train = [os.path.join(ROOTDIR_DATASET_NORMAL, f"rec{r}_20220811_rbtc_{1/frequency}s.metadata") for r in [0, 2, 3, 4]]

df_action_train, df_train, df_meta_train, action2int_train = get_metadata(filepath_csv_train, filepath_meta_train)    # read corresponding metadata
df_action_test, df_test, df_meta_test, action2int_test = get_metadata(filepath_csv_test, filepath_meta_test)

df_test['time'] = pd.to_datetime(df_test.time.astype('datetime64[ms]'), format ="%Y-%m-%dT%H:%M:%S.%f")

X_train = df_train.drop(['time'], axis=1, inplace=False)     # remove last column 'time' from dataset
X_collisions = df_test.drop(['time'], axis=1, inplace=False)

##### Get Collisions

In [8]:
timestamps_collisions = pd.read_excel(os.path.join(ROOTDIR_DATASET_NORMAL, "20220811_collisions_timestamp.xlsx"))
timestamps_collisions['Timestamp'] = timestamps_collisions['Timestamp'] - pd.to_timedelta(2, 'h')
# due to a time discrepancy, the time interval of the collisions should be anticipated of two hour
start_col = timestamps_collisions[timestamps_collisions['Inizio/fine'] == "i"][['Timestamp']].rename(columns={'Timestamp': 'start'}) # even indexes
end_col = timestamps_collisions[timestamps_collisions['Inizio/fine'] == "f"][['Timestamp']].rename(columns={'Timestamp': 'end'})     # odd indexes

start_col.reset_index(drop=True, inplace=True)  # reset the indexes
end_col.reset_index(drop=True, inplace=True)

df_collision = pd.concat([start_col, end_col], axis=1)  # concatenate start e end --> it becomes (key, start, end) 51 columns

# K-MEANS

In [ ]:
class KMeansAD(BaseEstimator, OutlierMixin):
    def __init__(self, k: int, window_size: int, padding_length: int):
        self.k = k                               # number of clusters
        self.window_size = window_size           # window size
        self.model = KMeans(n_clusters=k)        # model initialization
        self.padding_length = padding_length     # padding (length to add at the end of our score array)

    def _custom_reverse_windowing(self, scores: np.ndarray) -> np.ndarray:
        print("Reversing window-based scores to point-based scores:")
        print(f"Before reverse-windowing: scores.shape={scores.shape}")
        begins = np.arange(scores.shape[0]) * self.window_size                        # indexes of when window starts
        ends = begins + self.window_size                                              # indexes of when window ends
        unwindowed_length = scores.shape[0] * self.window_size + self.padding_length  # total length without windows
        mapped = np.full(unwindowed_length, fill_value=np.nan)                        # [nan nan nan ... nan nan nan]
        indices = np.unique(np.r_[begins, ends])                                      # [0 1 2 ... 34273 34274 34275], indexes of the scores
        for i, j in zip(indices[:-1], indices[1:]):                                   # arrange tuple [1, 2] [2, 3] ...
            if i >= 0 and j <= unwindowed_length:                                     # check if i is positive and j is lower than unwindowed_length
                window_indices = np.flatnonzero((begins <= i) & (j <= ends))          # it gives us the indexes in the window
                if len(window_indices) > 0:                                           # check if it's positive
                    mapped[i:j] = np.nanmean(scores[window_indices])                  # map the value between i and j computing the mean
        np.nan_to_num(mapped, copy = False)                                           # convert the nan values into number
        print(f"After reverse-windowing: scores.shape={mapped.shape}")                # we have our mapped point-based scores
        return mapped

    def fit(self, X: np.ndarray, y=None, preprocess=True) -> 'KMeansAD':
        self.model.fit(X)     # compute k-means clustering, y ignored
        return self

    def predict(self, X: np.ndarray, preprocess=True) -> np.ndarray:
        clusters = self.model.predict(X)    # predict cluster index for each sample
        diffs = np.linalg.norm(X - self.model.cluster_centers_[clusters], axis=1)
                # after assigning cluster labels, compute euclidean distance (L2 Norm) between every data point and the cluster center to whom they have been assigned to
                # np.linalg.norm: compute euclidean norm
                # axis = 1: specified that the norm must be calculated along axis = 1, which means that it is computed for each row (datapoint) individually
                # diff: array where each element represent the distance between the point and the cluster
                # these distances will be used to compute the anomaly scores. Furthers distances indicated that the point is far away from the cluster and could be considered as an anomaly

        return self._custom_reverse_windowing(diffs)

    def fit_predict(self, X, y=None) -> np.ndarray:
        self.fit(X, y, preprocess=False)
        return self.predict(X, preprocess=False)

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

def get_features(domain, df, frequency, window_size):
    cfg = tsfel.get_features_by_domain(domain)                     # receive the features by domain: 20 statistical, 14 temporal, 26 spectral
    extracted_features = tsfel.time_series_features_extractor(cfg,
                                                              df.select_dtypes(['number']),
                                                              fs=frequency,
                                                              window_size=window_size,
                                                              verbose=False)
    return extracted_features     # return an array of shape [windows, features]

In [ ]:
def compute_anomaly_scores(data, df_size, n_clusters = 10, window_size = 10, padding_length = 0):

  padding_length = df_size - (data.shape[0] * window_size)        # dimension you need to add

  detector = KMeansAD(n_clusters, window_size, padding_length)    # init the model
  start_time = time.time()
  anomaly_scores = detector.fit_predict(data)                             # return the scores
  print("--- %s seconds ---" % (time.time() - start_time))
  anomaly_scores = (anomaly_scores - np.min(anomaly_scores)) / (np.max(anomaly_scores) - np.min(anomaly_scores)) # Normalizing anomaly scores
  return anomaly_scores

In [ ]:
def pre_processing(data):  # pre-processing without highly correlated features

  data = data.drop((data.columns[data.isna().any()].tolist()), axis = 1) # remove nan values

  scaler = preprocessing.StandardScaler() # Normalizing_features
  scaler.fit(data)
  data = pd.DataFrame(scaler.transform(data), columns=data.columns)

  selector_variance = VarianceThreshold() # Remove zero-variance features
  selector_variance.fit(data)
  data = pd.DataFrame(selector_variance.transform(data), columns=data.columns.values[selector_variance.get_support()])

  return data

def features_extractions(window_size, n_clusters, domain):
    # window_size_spectral = 15
    # domain: # statistical, temporal, spectral
    features = get_features(domain, X_collisions, frequency, window_size) # (windows, features) (3427, 2200)
    features_norm = pre_processing(features)
    return features_norm

### Computing scores on test dataset

In [ ]:
features = features_extractions(window_size=10, n_clusters=15, domain='statistical')

In [ ]:
scores = compute_anomaly_scores(features, X_collisions.shape[0], n_clusters=15, window_size=10)

### Evaluate functions

In [9]:
# plot distribution and return the true_labels
def plot_hist(anomaly_scores, df_collision, df, title):
    index_anomaly = []      # anomalies' index
    idx = 0
    for _, row in df.iterrows():
        for _, collision_row in df_collision.iterrows():
            if (row['time'] >= collision_row['start']) and (row['time'] <= collision_row['end']):
                index_anomaly.append(idx)
        idx += 1
    true_labels = np.zeros_like(anomaly_scores)
    true_labels[index_anomaly] = 1
    logging.info(f"Anomalies detected: {int(true_labels.sum())}")
    anomaly_values = anomaly_scores[index_anomaly]
    normal_values = np.delete(anomaly_scores, index_anomaly)

    plt.hist(normal_values, bins=30, color="tab:blue", ec="dodgerblue", alpha=0.5, label='Normal')
    plt.hist(anomaly_values, bins=30, color='tab:red', ec="darkred", alpha=0.7, label='Anomalies')

    plt.xlabel('Values')
    plt.ylabel('Occurrencies')
    plt.legend(loc='upper right')
    plt.title(title)
    plt.savefig(f'/content/{title}.jpg')  # Modify the path and filename as needed
    plt.show()
    return true_labels

# compute f1, fB score, auc-roc, auc-pr
def compute_metrics(anomaly_scores_norm, df_test, y_true, th=None):
    tot_anomalies = y_true.sum()
    sens = list()           # recalls o tpr
    spec = list()
    fpr = list()
    f1 = list()
    f0_1= list()
    prec = list()
    cm_list = list()
    anomlay_indexes_dict = dict()
    acc_with_err = list()
    step = 0.01
    ths = np.arange(0, 1, step)
    if th is None:
        for threshold in tqdm(ths):
            anomalies_pred = anomaly_scores_norm > threshold
            tp = 0                                                          # true positive per quella threshold
            anomaly_indexes = list()
            for index, anomaly_pred in enumerate(anomalies_pred):
                if y_true[index] and anomaly_pred:
                    anomaly_indexes.append(index)
                    tp += 1

            cm_anomaly = np.zeros((2,2))
            n_sample = len(df_test)
            n_not_collision = n_sample - tot_anomalies
            n_detected = anomalies_pred.sum()

            fp = n_detected - tp
            fn = tot_anomalies - tp
            tn = n_not_collision - fp

            cm_anomaly[0, 0] = tn
            cm_anomaly[0, 1] = fp
            cm_anomaly[1, 0] = fn
            cm_anomaly[1, 1] = tp

            cm_list.append(cm_anomaly)
            recall = tp / (tp + fn)
            sens.append(recall)
            fpr.append(1 - tn /(tn + fp))
            precision = tp / (tp + fp)
            prec.append(precision)
            spec.append(tn /(tn + fp))
            f1.append(2 * tp / (2 * tp + fp + fn))
            f0_1.append((1 + 0.1**2) * tp / ((1 + 0.1**2) * tp +  0.1**2*fp + fn))
            cm_anomaly_norm = cm_anomaly.astype('float') / cm_anomaly.sum(axis=1)[:, np.newaxis]
            acc_with_err.append( (np.mean(np.diag(cm_anomaly_norm)), np.std(np.diag(cm_anomaly_norm))) )
            anomlay_indexes_dict[threshold] = anomaly_indexes

        f1_max = max(f1)
        f0_1_max = max(f0_1)
        max_index_f1 = f1.index(f1_max)
        max_index_f0_1 = f0_1.index(f0_1_max)
        th_f1_max = max_index_f1 * step
        th_f0_1_max = max_index_f0_1 * step
        print(f"f1: {f1_max} at th: {th_f1_max}")
        print(f"f0.1: {f0_1_max} at th: {th_f0_1_max}")
        print(f"AUC-PR: {metrics.average_precision_score(y_true, anomaly_scores_norm)}")
        print(f"AUC-ROC: {metrics.roc_auc_score(y_true, anomaly_scores_norm)}")
        return sens, fpr, th_f1_max
    else:
        df_anomaly = df_test.loc[np.array(anomaly_scores_norm > th)]
        tp = 0                                                          # true positive per quella threshold
        anomaly_indexes = list()
        anomalies_pred = anomaly_scores_norm > th

        for index, anomaly_pred in enumerate(anomalies_pred):
            if y_true[index] and anomaly_pred:
                anomaly_indexes.append(index)
                tp += 1

        cm_anomaly = np.zeros((2,2))
        n_sample = len(df_test)
        n_not_collision = n_sample - tot_anomalies
        n_detected = len(df_anomaly)

        fp = n_detected - tp
        fn = tot_anomalies - tp
        tn = n_not_collision - fp

        cm_anomaly[0, 0] = tn
        cm_anomaly[0, 1] = fp
        cm_anomaly[1, 0] = fn
        cm_anomaly[1, 1] = tp

        f1 = 2 * tp / (2 * tp + fp + fn)
        f0_1 = (1 + 0.1**2) * tp / ((1 + 0.1**2) * tp +  0.1**2*fp + fn)
        print(f"f1: {f1} at th: {th} for the test set")
        print(f"f0.1: {f0_1} at th: {th} for the test set")

# another way to compute true_labels
def create_true_labels(df_test, df_collision, scores):
    index_anomaly = []
    idx = 0
    for _, row in df_test.iterrows():    # prende la riga da df_validation
        for _, collision_row in df_collision.iterrows():  # prende la collision da df_collision
            if (row['time'] >= collision_row['start']) and (row['time'] <= collision_row['end']):
                index_anomaly.append(idx)         # salva l'indice
        idx += 1               # aumenta l'indice
    true_labels = np.zeros_like(scores)
    true_labels[index_anomaly] = 1
    logging.info(f"Anomalies detected: {int(true_labels.sum())}")
    return true_labels

# dataset divition for testing with validation
def dataset_div(X_collisions, anomaly_scores_norm, df_test):
  split = 0.9                                    # splitting value
  split_at = int(len(X_collisions) * split)      # elements

  asn_val = anomaly_scores_norm[split_at:]       # validation scores
  asn_col = anomaly_scores_norm[:split_at]       # test scores

  df_val = df_test.iloc[split_at:]
  df_col = df_test.iloc[:split_at]

  df_val = df_val[-asn_val.shape[0]:]
  df_col = df_col[-asn_col.shape[0]:]

  return df_val, df_col, asn_val, asn_col

# Testing on a single model

##### Uploading scores

In [10]:
with open('/content/kmeans_f10_clusters15_w10', "rb") as file:
      scores = pickle.load(file)

##### Computing true_labels and metrics

In [ ]:
true_labels = plot_hist(scores, df_collision, df_test, title='KMeans_distribution_f=10Hz')

In [ ]:
compute_metrics(scores, df_test, true_labels)

##### Testing with validation split

In [ ]:
df_val, df_col, asn_val, asn_col = dataset_div(X_collisions, scores, df_test)
true_labels_val = plot_hist(asn_val, df_collision, df_val, title='KMeans_Distribution_Val_f=10Hz')
_, _, th_f1_max = compute_metrics(asn_val, df_val, true_labels_val)
true_labels_test = plot_hist(asn_col, df_collision, df_col, title='KMeans_Distribution_Test_f=10Hz')
compute_metrics(asn_col, df_col, true_labels_test, th_f1_max)

##### Downloading scores

In [ ]:
anomaly_score = {
            'anomaly_scores_norm' : scores,
            'true_labels' : true_labels
        }

In [ ]:
with open('/content/drive/MyDrive/kmeans_f10_clusters15_w10.pkl', 'wb') as file:
    pickle.dump(anomaly_score, file)
files.download('/content/drive/MyDrive/kmeans_f10_clusters15_w10.pkl')